In [1]:
import numpy as np
import pandas as pd
import os
import glob
import codecs
import scispacy
import spacy
import sklearn
import warnings 

from collections import defaultdict, Counter
from sklearn.model_selection import KFold
# from negspacy.negation import Negex
from sklearn.svm import SVC, LinearSVC, NuSVC
from collections import defaultdict
from transformers import BertTokenizer
from nltk.corpus import stopwords

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:

  
warnings.filterwarnings(action = 'ignore') 
  
# print(model1.similarity("Hospice", "Immunology"))                               
# nlp0 = spacy.load("en_core_web_sm")
# negex = Negex(nlp0, language = "en_clinical")
# nlp0.add_pipe(negex, last=True)
# doc0 = nlp0("he is not an emotional eater")
import string

#string punctuation
punct = set(string.punctuation)
# for e in doc0.ents:
# 	print(e.text, e._.negex)

# Using scispacy with "en_core_sci_sm":
nlp = spacy.load("en_core_sci_sm")

# # Using scispacy with "en_core_sci_lg":
# nlp = spacy.load("en_core_sci_lg")

# # Using scispacy with "en_ner_bc5cdr_md:
# nlp = spacy.load("en_ner_bc5cdr_md")


def tokenize_doc(text):
    bow = defaultdict(float)
    temp = []
    doc = nlp(text)
    doc = doc.ents
    lowered_tokens = map(lambda t: str(t).lower(), doc)

    for token in lowered_tokens:
        # token = token.split(".,")
        # if (len(token) > 1):
        #     for j in token:
        #         j = j.split(":,")
        #         for i in j:
        #             bow[i] += 1.0
        #             temp.append(i)
        # else:
        #     for i in token:
        #         bow[i] += 1.0
        #         temp.append(i)

        temp.append(token)
    return temp

def tokenize_doc2(text):
    t = BertTokenizer.from_pretrained("bert-large-uncased-vocab.txt", lowercase=True)
    warnings.filterwarnings('ignore')
    output = t.tokenize(text)
    temp = []
    words = [w for w in output if w not in punct]
    for token in words:
        temp.append(token)
    return temp

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv("mtsamples.csv")

In [6]:
ids = data["transcription"]
data = data.drop_duplicates(subset = ["description","medical_specialty","sample_name","transcription"])
temp = [i for i in data.index if pd.isna(data["transcription"][i])]
data = data.drop(temp)

        

In [7]:
data = data.reset_index(drop=True)

In [8]:
ids = data["transcription"]
data = data.drop_duplicates(subset = ["description","medical_specialty","sample_name","transcription"])
temp = [i for i in data.index if pd.isna(data["transcription"][i])]

surgery = list(data[data["medical_specialty"] == ' Surgery'].index.values)
consult = list(data[data["medical_specialty"] == ' Consult - History and Phy.'].index.values)
cardiovascular = list(data[data["medical_specialty"] == ' Cardiovascular / Pulmonary'].index.values)
orthopedic = list(data[data["medical_specialty"] == ' Orthopedic'].index.values)
radiology = list(data[data["medical_specialty"] == ' Radiology'].index.values)

general = list(data[data["medical_specialty"] == ' General Medicine'].index.values)
gastroenterology = list(data[data["medical_specialty"] == ' Gastroenterology'].index.values)
neurology = list(data[data["medical_specialty"] == ' Neurology'].index.values)
soap = list(data[data["medical_specialty"] == ' SOAP / Chart / Progress Notes'].index.values)
obstetrics = list(data[data["medical_specialty"] == ' Obstetrics / Gynecology'].index.values)

urology = list(data[data["medical_specialty"] == ' Urology'].index.values)
discharge = list(data[data["medical_specialty"] == ' Discharge Summary'].index.values)
ent = list(data[data["medical_specialty"] == ' ENT - Otolaryngology'].index.values)
neurosurgery = list(data[data["medical_specialty"] == ' Neurosurgery'].index.values)
hematology = list(data[data["medical_specialty"] == ' Hematology - Oncology'].index.values)

ophthalmology = list(data[data["medical_specialty"] == ' Ophthalmology'].index.values)
Nephrology = list(data[data["medical_specialty"] == ' Nephrology'].index.values)


uniq_dict = {}

temp_list = surgery[76:] + consult[76:] + cardiovascular[76:] + orthopedic[76:] + radiology[76:] + general[76:] + gastroenterology[76:] + neurology[76:] + soap[76:] + obstetrics[76:] + urology[76:] + discharge[76:] + ent[76:] + neurosurgery[76:] + hematology[76:] + ophthalmology[76:] + Nephrology[76:]

In [9]:
print(temp_list)

[252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451,

In [10]:
train_data_temp = pd.read_csv("list.csv")
temp = [i for i in train_data_temp.index if pd.isna(train_data_temp["transcription"][i])]
train_data_temp = train_data_temp.drop(temp)

In [11]:
print(train_data_temp['transcription'][0])

PREOPERATIVE DIAGNOSES:,1.  Hallux rigidus, left foot.,2.  Elevated first metatarsal, left foot.,POSTOPERATIVE DIAGNOSES:,1.  Hallux rigidus, left foot.,2.  Elevated first metatarsal, left foot.,PROCEDURE PERFORMED:,1.  Austin/Youngswick bunionectomy with Biopro implant.,2.  Screw fixation, left foot.,HISTORY: , This 51-year-old male presents to ABCD General Hospital with the above chief complaint.  The patient states that he has had degenerative joint disease in his left first MPJ for many years that has been progressively getting worse and more painful over time.  The patient desires surgical treatment.,PROCEDURE IN DETAIL: , An IV was instituted by the Department of Anesthesia in the preoperative holding area.  The patient was transported from the operating room and placed on the operating room table in the supine position with the safety belt across his lap.  Copious amount of Webril was placed around the left ankle followed by a blood pressure cuff.  After adequate sedation by the

In [12]:
print(tokenize_doc(train_data_temp['transcription'][0])  )

['preoperative', 'diagnoses:,1', 'hallux rigidus', 'left foot.,2', 'elevated', 'metatarsal', 'left foot.,postoperative', 'diagnoses:,1', 'hallux rigidus', 'left foot.,2', 'elevated', 'metatarsal', 'left foot.,procedure performed:,1', 'austin/youngswick bunionectomy', 'biopro implant.,2', 'screw fixation', 'left foot.,history', 'male', 'abcd', 'general hospital', 'chief complaint', 'patient', 'degenerative joint disease', 'left', 'mpj', 'years', 'time', 'patient desires', 'surgical treatment.,procedure', 'detail', 'iv', 'department of anesthesia', 'preoperative', 'holding area', 'patient', 'transported', 'operating room', 'operating room', 'supine', 'position', 'safety', 'belt', 'copious', 'amount', 'webril', 'left ankle', 'blood pressure', 'adequate', 'sedation', 'department of anesthesia', 'marcaine plain', 'injected', 'mayo-type block', 'foot', 'prepped', 'draped', 'usual', 'sterile orthopedic fashion', 'foot', 'operating', 'table', 'exsanguinated', 'esmarch bandage', 'pneumatic ankl

In [13]:
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer


kf = KFold(n_splits=12, random_state=None, shuffle=False)
kf.get_n_splits(data)

Tfidf_vect = TfidfVectorizer(tokenizer=tokenize_doc)
# Tfidf_vect = TfidfVectorizer(tokenizer=tokenize_doc2)
features  = Tfidf_vect.fit(train_data_temp['transcription'])


In [14]:
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

nsvc = NuSVC(nu=0.0001, probability=True)
lsvc = LinearSVC(tol = 1e-5, C = 0.0001, random_state=500000)

In [20]:
x_train, x_test = train_data_temp['transcription'], data['transcription'][temp_list]
y_train, y_test = train_data_temp['medical_specialty'], data['medical_specialty'][temp_list]
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)
lsvc.fit(Train_X_Tfidf,y_train)
predictions_lSVM = lsvc.predict(Test_X_Tfidf)
print(metrics.classification_report(predictions_lSVM, y_test.to_numpy(), digits=5))

                                precision    recall  f1-score   support

    Cardiovascular / Pulmonary    0.07458   0.36667   0.12394        60
    Consult - History and Phy.    0.29773   0.47985   0.36746       273
             Discharge Summary    0.87500   0.10108   0.18123       277
          ENT - Otolaryngology    0.50000   0.15873   0.24096        63
              Gastroenterology    0.60135   0.26488   0.36777       336
              General Medicine    0.32787   0.24793   0.28235       242
         Hematology - Oncology    0.35714   0.05155   0.09009        97
                    Nephrology    0.40000   0.01493   0.02878       134
                     Neurology    0.57143   0.37004   0.44920       227
                  Neurosurgery    0.94444   0.04106   0.07870       414
       Obstetrics / Gynecology    0.37975   0.32967   0.35294        91
                 Ophthalmology    0.57143   0.05882   0.10667        68
                    Orthopedic    0.47670   0.38776   0.42765  

In [17]:
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data):
    # print("TRAIN:", len(train_index), "TEST:", len(test_index))
    # for train_index, test_index in kf.split(data):
    x_train = []
    y_train = []

    x_train, x_test = data['transcription'][train_index], data['transcription'][test_index]
    y_train, y_test = data['medical_specialty'][train_index], data['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
#     nsvc.fit(Train_X_Tfidf,y_train) 
    lsvc.fit(Train_X_Tfidf,y_train) 
    
#     predictions_nuSVM = nsvc.predict(Test_X_Tfidf)
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())


    print(accuracy_score(y_test, predictions_nuSVM)*100)
#     total += accuracy_score(y_test, predictions_nuSVM)*100
#     count += 1

# print(count)
# print(total/count)
print(total/count)
print(res)
    


                             precision    recall  f1-score   support

       Allergy / Immunology    0.00000   0.00000   0.00000         0
                 Bariatrics    0.00000   0.00000   0.00000         0
 Cardiovascular / Pulmonary    0.00000   0.00000   0.00000         0
                  Dentistry    0.00000   0.00000   0.00000         0
           General Medicine    0.00000   0.00000   0.00000         0
                  Neurology    0.00000   0.00000   0.00000         0
                    Surgery    1.00000   0.57488   0.73006       414
                    Urology    0.00000   0.00000   0.00000         0

                   accuracy                        0.57488       414
                  macro avg    0.12500   0.07186   0.09126       414
               weighted avg    1.00000   0.57488   0.73006       414

57.48792270531401
              precision    recall  f1-score   support

     Surgery    1.00000   1.00000   1.00000       414

    accuracy                        1.000

In [18]:
x_train, x_test = data['transcription'][0:5], data['transcription'][0:5]
y_train, y_test = data['medical_specialty'][0:5], data['medical_specialty'][0:5]
                                                  
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)

nsvc.fit(Train_X_Tfidf,y_train)   
#classifier2.fit(Train_X_Tfidf,y_train)

predictions_nuSVM = nsvc.predict(Test_X_Tfidf)
print(accuracy_score(predictions_nuSVM, y_test)*100)

100.0


In [ ]:
print(len(Tfidf_vect.vocabulary_))

In [ ]:
print(Tfidf_vect.vocabulary_)

In [ ]:
print(Tfidf_vect.get_feature_names())